In [1]:
import math

def haversine(coord1, coord2):
    # Rayon de la Terre en kilomètres
    R = 6371.0
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat / 2)**2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

In [8]:
import networkx as nx

def construire_graphe(city_data):
    G = nx.Graph()
    
    # Créer un dictionnaire pour accéder rapidement aux coordonnées des communes
    communes = {item['commune']: item['coordonnees'] for item in city_data}
    
    # Ajouter chaque commune comme nœud avec ses coordonnées
    for commune, coord in communes.items():
        G.add_node(commune, pos=coord)
    
    # Ajouter les arêtes pondérées par la distance
    for commune, coord in communes.items():
        for autre_commune, autre_coord in communes.items():
            if commune != autre_commune:
                dist = haversine(coord, autre_coord)
                G.add_edge(commune, autre_commune, weight=dist)
    
    return G

In [10]:
# Fonction pour trouver le chemin le plus court entre deux communes
def chemin_le_plus_court(G, source, destination):
    chemin = nx.shortest_path(G, source=source, target=destination, weight='weight')
    distance_totale = nx.shortest_path_length(G, source=source, target=destination, weight='weight')
    return chemin, distance_totale


In [16]:
import json

with open('assets/data.json', 'r') as file:
    data = json.load(file)
city_data = [{'commune': item['COMMUNE'].lower(), 'coordonnees': item['Geo Shape']['coordinates']} for item in data]

In [17]:
G = construire_graphe(city_data)

In [29]:
source = "lille"
destination = "marseille"

In [30]:
chemin, distance = chemin_le_plus_court(G, source, destination)

In [31]:
print("Chemin le plus court:", chemin)
print("Distance totale:", distance, "km")

Chemin le plus court: ['lille', 'marseille']
Distance totale: 855.1126297477794 km
